<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Imp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
You have a dataset of transactions with the following fields:
transaction_id (integer): Unique ID for each transaction.
user_id (integer): ID of the user performing the transaction.
transaction_amount (float): Amount of the transaction.
transaction_date (string): Date of the transaction in yyyy-MM-dd format.
From this dataset, perform the following operations:

For each user, calculate the average gap in days between consecutive transactions.
Identify the user with the largest average gap.

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
data = [
 (1, 101, 500.0, "2024-01-01"),
 (2, 102, 200.0, "2024-01-02"),
 (3, 101, 300.0, "2024-01-03"),
 (4, 103, 100.0, "2024-01-04"),
 (5, 102, 400.0, "2024-01-05"),
 (6, 103, 600.0, "2024-01-06"),
 (7, 101, 200.0, "2024-01-07"),
]
columns = ["transaction_id", "user_id", "transaction_amount", "transaction_date"]

In [8]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.window import Window

spark=SparkSession.builder.appName("learning").getOrCreate()

data= [
 (1, 101, 500.0, "2024-01-01"),
 (2, 102, 200.0, "2024-01-02"),
 (3, 101, 300.0, "2024-01-03"),
 (4, 103, 100.0, "2024-01-04"),
 (5, 102, 400.0, "2024-01-05"),
 (6, 103, 600.0, "2024-01-06"),
 (7, 101, 200.0, "2024-01-07"),
]

schema=["transaction_id", "user_id", "transaction_amount", "transaction_date"]

df=spark.createDataFrame(data,schema)

df.show()

df1=df.withColumn("transaction_date",to_date(col("transaction_date"),'yyyy-MM-dd'))

df1.printSchema()

window_spec=Window.partitionBy('user_id').orderBy('transaction_date')

df2=df1.withColumn('prev_date',lag(col('transaction_date')).over(window_spec))

df2.show()

df3=df2.withColumn("day_gap",datediff(col('transaction_date'),col('prev_date')))

df3.show()

df4=df3.groupBy('user_id').agg(avg('day_gap').alias('avg_day_gap'))

df4.show()

window_spec1=Window.orderBy(col('avg_day_gap').desc())

df5=df4.withColumn('rank',dense_rank().over(window_spec1)).filter(col('rank')==1)

df5.show()



+--------------+-------+------------------+----------------+
|transaction_id|user_id|transaction_amount|transaction_date|
+--------------+-------+------------------+----------------+
|             1|    101|             500.0|      2024-01-01|
|             2|    102|             200.0|      2024-01-02|
|             3|    101|             300.0|      2024-01-03|
|             4|    103|             100.0|      2024-01-04|
|             5|    102|             400.0|      2024-01-05|
|             6|    103|             600.0|      2024-01-06|
|             7|    101|             200.0|      2024-01-07|
+--------------+-------+------------------+----------------+

root
 |-- transaction_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- transaction_amount: double (nullable = true)
 |-- transaction_date: date (nullable = true)

+--------------+-------+------------------+----------------+----------+
|transaction_id|user_id|transaction_amount|transaction_date| prev_date|
+